In [1]:
import sys
sys.path.append("d:\\FundResearch\\")
import quant_utils.data_moudle as dm
import pandas as pd
from quant_utils.constant import DB_CONN_JJTG_DATA,DB_CONN_DATAYES, BARRA_SW21_FACTOR_NAME_DICT
from dateutil.parser import parse

季报2固收及固收加杠杆券种权益仓位变动

In [2]:
query_sql = """
WITH t ( REPORT_DATE, LEVEL1, LEVEL2, F_PRT_BONDTONAV, F_PRT_COVERTBONDTONAV, F_PRT_INTERESETBONDTONAV, F_PRT_CREDITBONDTONAV, EQUITY_IN_NA, HK_in_na ) AS (
	SELECT
	CASE
			grouping ( a.REPORT_DATE ) 
			WHEN 1 THEN
			'总计' ELSE a.REPORT_DATE 
		END,
	CASE
			grouping ( b.LEVEL_1 ) 
			WHEN 1 THEN
			'总计' ELSE b.LEVEL_1 
		END,
	CASE
			grouping ( b.LEVEL_2 ) 
			WHEN 1 THEN
			'--' ELSE b.LEVEL_2 
		END,
		avg( a.F_PRT_BONDTONAV ),
		avg( a.F_PRT_COVERTBONDTONAV ),
		avg( `a`.`F_PRT_GOVBONDTONAV` + `a`.`F_PRT_LOCALGOVBONDTONAV` + `a`.`F_PRT_CTRBANKBILLTONAV` + `a`.`F_PRT_POLIFINANBDTONAV` ),
		avg( `a`.`F_PRT_BONDTONAV` - `a`.`F_PRT_COVERTBONDTONAV` - `a`.`F_PRT_GOVBONDTONAV` - `a`.`F_PRT_LOCALGOVBONDTONAV` - `a`.`F_PRT_CTRBANKBILLTONAV` - a.F_PRT_POLIFINANBDTONAV ),
		avg( a.EQUITY_MARKET_VALUE / a.NET_ASSET * 100 ),
		avg( a.F_PRT_HKSTOCKVALUETONAV ) 
	FROM
		fund_asset_own a
		JOIN fund_type_own b ON b.REPORT_DATE = a.REPORT_DATE 
		AND a.TICKER_SYMBOL = b.TICKER_SYMBOL 
	WHERE
		b.LEVEL_1 IN ( '固收', '固收+' ) 
	GROUP BY
		a.REPORT_DATE,
		b.LEVEL_1,
		b.LEVEL_2 WITH ROLLUP 
	HAVING
		( grouping ( a.REPORT_DATE ) <> 1 ) 
	) SELECT
	REPORT_DATE as '报告期',
	level1 as '知己内部分类:一级',
	level2 as '知己内部分类:二级',
	round(F_PRT_BONDTONAV,2) + round(EQUITY_IN_NA,2) as '杠杆',
	round(F_PRT_COVERTBONDTONAV,2) as '可转债占净值',
	round(F_PRT_INTERESETBONDTONAV,2) as '利率债占净值',
	round(F_PRT_CREDITBONDTONAV,2) as '信用债占净值',
	round(EQUITY_IN_NA,2) as '权益占净值',
    round(EQUITY_IN_NA + 0.5*F_PRT_COVERTBONDTONAV,2) as '风险暴露'
FROM
	t 
WHERE
	level2 IS NOT NULL 
	and level1 != '总计' and level2 != '--'
	and report_date >= '20191231'
ORDER BY
	level1,
	level2,
	REPORT_DATE;
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df_leverage = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='杠杆')
df_insteret = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='利率债占净值')
df_credit = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='信用债占净值')
df_stock = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='权益占净值')
df_risk_exposure = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='风险暴露')

In [3]:
df_risk_exposure

知己内部分类:二级,中长期纯债型基金,可转债,固收+(0-5),固收+(15-25),固收+(25-35),固收+(35+),固收+(5-15),短期纯债型基金
报告期,,,,,,,,
2020-03-31,0.02,51.17,3.47,21.81,32.12,40.28,11.90,0.00
2020-06-30,0.02,53.61,2.84,21.26,30.57,41.61,12.07,0.00
2020-09-30,0.01,53.07,3.41,21.60,28.92,39.30,11.70,0.00
2020-12-31,0.02,53.42,3.16,21.53,29.47,37.97,12.36,0.01
2021-03-31,0.03,52.45,3.14,19.56,27.39,35.66,10.50,0.01
2021-06-30,0.02,50.42,3.18,19.70,28.38,41.02,11.25,0.01
2021-09-30,0.02,49.51,3.25,19.85,28.86,37.26,10.68,0.00
2021-12-31,0.02,51.03,3.20,20.77,30.80,40.70,12.18,0.01
2022-03-31,0.02,54.63,3.01,19.16,29.82,42.98,10.02,0.01


权益基金1仓位

In [4]:
query_sql = """
WITH t ( REPORT_DATE, level1,level2,EQUITY_IN_NA, HK_in_na, A_in_na ) AS (
	SELECT
	CASE
			grouping ( a.REPORT_DATE ) 
			WHEN 1 THEN
			'总计' ELSE a.REPORT_DATE 
		END,
	CASE
			grouping ( b.LEVEL_1 ) 
			WHEN 1 THEN
			'总计' ELSE b.LEVEL_1 
		END,
	CASE
			grouping ( b.LEVEL_2 ) 
			WHEN 1 THEN
			'--' ELSE b.LEVEL_2 
		END,
		avg( a.EQUITY_MARKET_VALUE / a.NET_ASSET * 100 ),
		avg( a.F_PRT_HKSTOCKVALUETONAV ),
		avg( a.EQUITY_MARKET_VALUE / a.NET_ASSET * 100 - a.F_PRT_HKSTOCKVALUETONAV)
	FROM
		fund_asset_own a
		JOIN fund_type_own b ON b.REPORT_DATE = a.REPORT_DATE 
		AND a.TICKER_SYMBOL = b.TICKER_SYMBOL 
	WHERE
		b.LEVEL_1 = '主动权益' 
	GROUP BY
		a.REPORT_DATE,
		b.LEVEL_1,
		b.LEVEL_2 WITH ROLLUP 
	HAVING
		( grouping ( a.REPORT_DATE ) <> 1 ) 
	) SELECT
	REPORT_DATE as '报告期',
	level1 as '知己内部分类:一级',
	level2 as '知己内部分类:二级',
	round(EQUITY_IN_NA,2) as '仓位',
	round(HK_in_na,2) as '港股仓位',
	round(A_in_na,2) as 'A股仓位'
FROM
	t 
WHERE
	level2 IS NOT NULL and level1 != '总计'
	and report_date >= '20191231'
ORDER BY
	level1,
	level2,
	REPORT_DATE;
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [5]:
df

,报告期,知己内部分类:一级,知己内部分类:二级,仓位,港股仓位,A股仓位
0,2020-03-31,主动权益,--,84.59,2.40,82.19
1,2020-06-30,主动权益,--,87.01,2.34,84.67
2,2020-09-30,主动权益,--,85.83,3.08,82.80
3,2020-12-31,主动权益,--,87.45,3.80,83.64
4,2021-03-31,主动权益,--,84.64,4.75,79.91
5,2021-06-30,主动权益,--,85.81,5.16,80.70
6,2021-09-30,主动权益,--,85.31,4.37,80.95
7,2021-12-31,主动权益,--,86.91,4.84,82.07
8,2022-03-31,主动权益,--,84.43,4.81,79.66
9,2022-06-30,主动权益,--,87.04,5.62,81.49


权益基金2-前十大占比

In [6]:
query_sql = """
WITH t AS (
	SELECT
		REPORT_DATE,
		FUND_ID,
		HOLDING_TICKER_SYMBOL,
		HOLDING_SEC_SHORT_NAME,
		RATIO_IN_NA,
        MARKET_VALUE,
		ROW_NUMBER() over ( PARTITION BY REPORT_DATE, FUND_ID, SECURITY_TYPE ORDER BY RATIO_IN_NA DESC ) AS NUMBER_N 
	FROM
		fund_holdings 
	WHERE
		REPORT_DATE >= '20141231' 
		AND SECURITY_TYPE = 'E' 
	ORDER BY
		REPORT_DATE,
		FUND_ID,
		NUMBER_N 
	) SELECT
	b.TICKER_SYMBOL,
	b.SEC_SHORT_NAME,
	t.*,
	d.PREV_TRADE_DATE,
	c.LEVEL_1,
	c.LEVEL_2
FROM
	t
	JOIN fund_info b ON b.FUND_ID = t.FUND_ID and b.IS_MAIN = 1
	JOIN fund_type_own c ON c.TICKER_SYMBOL = b.TICKER_SYMBOL 
	AND c.report_date = t.report_date 
	AND c.LEVEL_1 = '主动权益'
	join md_trade_cal d on d.CALENDAR_DATE = t.REPORT_DATE and d.EXCHANGE_CD = 'XSHG'
WHERE t.NUMBER_N <= 10 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [40]:
df_sum = df.groupby(by=['PREV_TRADE_DATE', 'FUND_ID', 'TICKER_SYMBOL', 'SEC_SHORT_NAME', 'LEVEL_1', 'LEVEL_2']).sum().reset_index()
df_top10_sum_mean = df_sum.groupby(by=['PREV_TRADE_DATE', 'LEVEL_1', 'LEVEL_2'])['RATIO_IN_NA'].mean().reset_index()
df_top10_sum_mean1 = df_top10_sum_mean.pivot_table(
    index='PREV_TRADE_DATE', values="RATIO_IN_NA", columns='LEVEL_1'
)

In [166]:
df_sum_mean = (
    df.groupby(
        by=['PREV_TRADE_DATE', 'HOLDING_TICKER_SYMBOL', 'HOLDING_SEC_SHORT_NAME']
    ).sum()
    .reset_index()
    .sort_values(by=['PREV_TRADE_DATE', "RATIO_IN_NA"], ascending=False)
)

df_sum_mean['rank'] = df_sum_mean.groupby(
        by=['PREV_TRADE_DATE']
    )["MARKET_VALUE"].rank(ascending=False)

df_sum = df_sum_mean.groupby(
        by=['PREV_TRADE_DATE']
    )["MARKET_VALUE"].sum().reset_index().rename(columns={"MARKET_VALUE": "MARKET_VALUE_SUM"})
df_sum_mean=  df_sum_mean.merge(df_sum)
df_sum_mean["比例"] = (df_sum_mean['MARKET_VALUE']/df_sum_mean['MARKET_VALUE_SUM'])*100

权益基金-公募基金前50/100大占比

In [91]:
df_sum_mean.query("rank <= 100").groupby("PREV_TRADE_DATE")['比例'].sum()

PREV_TRADE_DATE
2014-12-30    61.499295
2015-03-30    53.548001
2015-06-29    44.792653
2015-09-29    48.374712
2015-12-30    40.665114
2016-03-30    40.877882
2016-06-29    39.420889
2016-09-29    40.493261
2016-12-30    38.118547
2017-03-30    43.440195
2017-06-29    53.064334
2017-09-29    58.308881
2017-12-29    65.935426
2018-03-30    64.511934
2018-06-29    64.115830
2018-09-28    64.978720
2018-12-28    61.654067
2019-03-29    64.665600
2019-06-28    71.015066
2019-09-27    70.353856
2019-12-30    69.041477
2020-03-30    67.030028
2020-06-29    67.838032
2020-09-29    69.852704
2020-12-30    74.874446
2021-03-30    72.439008
2021-06-29    71.593602
2021-09-29    65.091270
2021-12-30    63.202569
2022-03-30    61.606522
2022-06-29    63.034054
2022-09-29    59.667935
2022-12-30    57.591460
Name: 比例, dtype: float64

权益基金-前十大风格

In [167]:
barra_exposure_sql = """
SELECT
	a.*
FROM
	dy1d_exposure_cne6 a
	join (SELECT DISTINCT
		quarter_end_date 
	FROM
		md_trade_cal 
WHERE
	EXCHANGE_CD = 'XSHG' 
	AND quarter_end_date BETWEEN '20141230' 
AND '20221230') t on a.TRADE_DATE = t.quarter_end_date
"""
barra_exposure = DB_CONN_DATAYES.exec_query(barra_exposure_sql).drop(columns=['ID'])
barra_exposure["TRADE_DATE"] = barra_exposure["TRADE_DATE"].apply(lambda s: parse(s).strftime("%Y-%m"))

In [168]:
barra_exposure = barra_exposure.dropna(axis=1)

In [169]:
barra_exposure.loc[:, "BETA": "COUNTRY"] = barra_exposure.loc[:, "BETA": "COUNTRY"].astype("float")

In [170]:
df_sum_mean["PREV_TRADE_DATE1"] = df_sum_mean["PREV_TRADE_DATE"].apply(lambda s: s.strftime("%Y-%m"))

In [135]:
(
    df_sum_mean.merge(
        barra_exposure, 
        left_on=['HOLDING_TICKER_SYMBOL', "PREV_TRADE_DATE1"], right_on=['TICKER_SYMBOL', "TRADE_DATE"]
    ).groupby(by='PREV_TRADE_DATE1')
    .mean()
    .rename(columns=BARRA_SW21_FACTOR_NAME_DICT)
    .to_excel("d:/barra.xlsx")
)

In [199]:
df_barra = df_sum_mean.merge(
        barra_exposure, 
        left_on=['HOLDING_TICKER_SYMBOL', "PREV_TRADE_DATE1"], right_on=['TICKER_SYMBOL', "TRADE_DATE"]
    )

In [200]:
col = df_barra.columns

In [202]:
for column in col[13:-2]:
    df_barra.loc[:, column] = df_barra.loc[:, column]*df_barra.loc[:, "比例"]

In [203]:
df1 = (df_barra.groupby(by='PREV_TRADE_DATE1')
    .sum()

)

In [204]:
for column in col[13:-2]:
    df1.loc[:, column] = df1.loc[:, column]/df1.loc[:, "比例"]

In [197]:
df1.rename(columns=BARRA_SW21_FACTOR_NAME_DICT).to_excel("D:/barra.xlsx")

In [201]:
col[13]

'BETA'